In [19]:
import asyncio
from telethon import TelegramClient
from telethon.sessions import StringSession
import os
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Telegram API Credentials
api_id = '26853954'
api_hash = '33fdd5d76a86b127138125c34b08718'
phone_number = '+919739424247'

# Create a custom session file path
SESSION_FILE = os.path.join(os.getcwd(), 'telegram_session.session')

async def create_telegram_client():
    try:
        # Remove existing session file if it's causing issues
        if os.path.exists(SESSION_FILE):
            os.remove(SESSION_FILE)
        
        # Create client with a specific session file
        client = TelegramClient(SESSION_FILE, api_id, api_hash)
        
        # Connect and handle authentication
        await client.start(phone=phone_number)
        
        return client
    
    except Exception as e:
        print(f"Error creating Telegram client: {e}")
        return None

In [20]:
async def main():
    # Create Telegram client
    client = await create_telegram_client()
    
    if not client:
        print("Failed to create Telegram client")
        return
    
    try:
        # List of Telegram channels
        stock_channels = [
            '@STOCKGAINERSS',
            '@hindustan_trader_9',
            '@bloomberg'
        ]
        
        # Collect messages
        collected_messages = []
        
        for channel_username in stock_channels:
            try:
                # Get channel entity
                channel = await client.get_entity(channel_username)
                
                # Fetch messages
                messages = await client.get_messages(channel, limit=1000)
                
                # Process messages
                for message in messages:
                    if message and hasattr(message, 'text') and message.text:
                        collected_messages.append({
                            'channel': channel_username,
                            'message_text': message.text,
                            'date': message.date
                        })
            
            except Exception as channel_error:
                print(f"Error in channel {channel_username}: {channel_error}")
        
        # Process collected messages
        if collected_messages:
            # Your existing data processing code here
            process_messages(collected_messages)
        else:
            print("No messages collected")
    
    except Exception as e:
        print(f"General error: {e}")
    
    finally:
        # Ensure client is properly closed
        await client.disconnect()

def process_messages(collected_messages):
    # Sentiment Analysis
    nltk.download('vader_lexicon', quiet=True)
    sia = SentimentIntensityAnalyzer()
    
    # Convert to DataFrame
    messages_df = pd.DataFrame(collected_messages)
    
    # Compute Sentiment
    messages_df['sentiment_score'] = messages_df['message_text'].apply(
        lambda x: sia.polarity_scores(str(x))['compound']
    )
    
    # Feature Engineering
    messages_df['text_length'] = messages_df['message_text'].str.len()
    messages_df['word_count'] = messages_df['message_text'].str.split().str.len()
    
    # Prepare Features
    features = messages_df[['sentiment_score', 'text_length', 'word_count']]
    
    # Create Target Variable
    messages_df['stock_movement'] = np.where(
        messages_df['sentiment_score'] > 0, 1, 0
    )
    
    target = messages_df['stock_movement']
    
    # Split Data
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.2, random_state=42
    )
    
    # Scale Features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train Model
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X_train_scaled, y_train)
    
    # Predictions
    y_pred = rf_classifier.predict(X_test_scaled)
    
    # Print Results
    print(classification_report(y_test, y_pred))

In [21]:
await main()

Please enter the code you received:  16302
Please enter your password:  ········


Signed in successfully as Karthikm; remember to not break the ToS or you will risk an account ban!
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       279
           1       1.00      1.00      1.00       228

    accuracy                           1.00       507
   macro avg       1.00      1.00      1.00       507
weighted avg       1.00      1.00      1.00       507

